In [3]:

from src.experiment import halfchetah_experiment


halfchetah_experiment({'mujoco': 1.0, 'brax': 0})

/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/gymnasium/envs/registration.py:519: DeprecationWarning: WARN: The environment HalfCheetah-v4 is out of date. You should consider upgrading to version `v5`.
  logger.deprecation(


✅ Created mujoco environment: HalfCheetah-v4
Initializing fast Brax environment: halfcheetah
✅ Fast Brax environment initialized with JIT compilation
  Observation space: Box(-inf, inf, (17,), float32)
  Action space: Box(-1.0, 1.0, (6,), float32)
✅ Created brax environment: BraxHalfCheetah-v0
✅ Created 2 environment types for sequential training.


/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)



Episode 1/500 Summary:
mujoco:  Mean Reward: -10.516  Mean Steps: 1000.0  Avg Duration: 0.057s
brax:  Mean Reward: 42.380  Mean Steps: 1000.0  Avg Duration: 5.418s

Episode 11/500 Summary:
mujoco:  Mean Reward: -162.875  Mean Steps: 1000.0  Avg Duration: 0.104s
brax:  Mean Reward: -201.353  Mean Steps: 1000.0  Avg Duration: 0.992s

Episode 21/500 Summary:
mujoco:  Mean Reward: 1278.320  Mean Steps: 1000.0  Avg Duration: 0.195s
brax:  Mean Reward: -919.885  Mean Steps: 1000.0  Avg Duration: 1.061s

Episode 31/500 Summary:
mujoco:  Mean Reward: 1777.084  Mean Steps: 1000.0  Avg Duration: 0.221s
brax:  Mean Reward: -1090.295  Mean Steps: 1000.0  Avg Duration: 0.994s

Episode 41/500 Summary:
mujoco:  Mean Reward: 2360.001  Mean Steps: 1000.0  Avg Duration: 0.186s
brax:  Mean Reward: -732.955  Mean Steps: 1000.0  Avg Duration: 0.929s

Episode 51/500 Summary:
mujoco:  Mean Reward: 2862.255  Mean Steps: 1000.0  Avg Duration: 0.178s
brax:  Mean Reward: -504.865  Mean Steps: 1000.0  Avg Durati

Traceback (most recent call last):
  File "/Users/felixpasewaldt/Documents/Dev/hysarl/src/sequentiell/pipeline.py", line 141, in run
    self.stats.export_to_hdf5(download_local=True, save_to_drive=True)
  File "/Users/felixpasewaldt/Documents/Dev/hysarl/src/sequentiell/stats.py", line 277, in export_to_hdf5
    'timestamp', data=np.string_(timestamp_str))
  File "/Users/felixpasewaldt/opt/anaconda3/envs/hysarl/lib/python3.10/site-packages/numpy/__init__.py", line 400, in __getattr__
    raise AttributeError(
AttributeError: `np.string_` was removed in the NumPy 2.0 release. Use `np.bytes_` instead.


✅ All training statistics exported to logs/training_stats_20250325_204445_*.csv
✅ Closed mujoco environment
✅ Closed brax environment


AttributeError: 'TrainingPipeline' object has no attribute 'dashboard'